<a href="https://colab.research.google.com/github/shruthibanothu/learning-sql/blob/main/predict_diabates_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
df=pd.read_csv("/content/diabetes.csv")

In [ ]:
df.head(5)

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.info()

In [ ]:
df['Outcome'].unique()

In [ ]:
df.describe()

In [ ]:
zero_col =['Glucose', 'BloodPressure', 'SkinThickness','Insulin', 'BMI']
(df[zero_col]==0).sum()

In [ ]:
for col in zero_col:
    df[col]=df[col].replace(0, df[col].median())



In [ ]:
(df[zero_col]==0).sum()

In [ ]:
df['Outcome'].value_counts()

sns.countplot(x='Outcome',data=df)
plt.title("Diabetes outcomes !")
plt.show()

In [ ]:
df.hist(figsize=(30,15))
plt.show()

In [ ]:
sns.boxplot(x='Outcome', y='Glucose', data=df)
plt.show()

sns.boxplot(x='Outcome', y='BMI', data=df)
plt.show()

sns.boxplot(x='Outcome', y='Age', data= df)
plt.show()


In [ ]:
plt.figure(figsize=(15,7))
sns.heatmap(df.corr(), annot=True)
plt.title("correlated hestmap")
plt.show()

** insights**
glucose has more correlation with outcome
Age and BMI are also has moderate correlation with outcome



**BASELINE MODELS**

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
df['Pregnancies'].unique()

In [ ]:

def base_model(row):
  if ((row['Glucose']> 140) and (row['BMI']>30)):
    return 1
  else :
    return 0
df['base_prediction_1']=df.apply(base_model, axis=1)
print(classification_report(df['Outcome'], df['base_prediction_1']))

def base_model_1(row):
  if((row['Age'] > 30) and (row['BMI']>30)):
    return 1
  else :
    return 0
df['base_prediction_2']=df.apply(base_model_1, axis=1)
print(classification_report(df['Outcome'], df['base_prediction_2']))

def base_model_2(row):
  if ((row['Pregnancies']> 10) and (row['Age']>20)):
    return 1
  else :
    return 0
df['base_prediction_3']=df.apply(base_model_2, axis=1)
print(classification_report(df['Outcome'], df['base_prediction_3']))



In [ ]:
df

## FIRST MODEL USING KERAS

In [ ]:
!pip install keras

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
x=df[['Pregnancies','Glucose', 'BloodPressure']]
y=df['Outcome']


In [ ]:
x

In [ ]:
y

In [ ]:
x_train,x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42)

scaler =StandardScaler()
x_train=scaler.fit_transform(x_train)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model_1 = Sequential([
    Dense(8, activation='relu', input_shape=(3,)),
    Dense(1, activation='sigmoid')
])

model_1.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])
history=model_1.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test,y_test), verbose=1)


In [ ]:
loss_1, acc_1 = model_1.evaluate(x_test, y_test)
print("Model 1 Accuracy:", acc_1)

## BUILD SECOND MODEL

In [ ]:
df.drop(columns=['base_prediction_1','base_prediction_2','base_prediction_3'], axis=1 ,inplace=True)

In [ ]:
x1=df.drop('Outcome', axis=1)
y=df['Outcome']
print(x1.shape,y.shape)

In [ ]:
x1_train,x1_test,y_train,y_test=train_test_split(x1,y,test_size=0.2, random_state=42)

scaler=StandardScaler()
x1_train=scaler.fit_transform(x1_train)
x_test=scaler.transform(x1_test)

In [ ]:
model_2= Sequential([
    Dense(16, activation='relu', input_dim=8),
    Dense(16, activation = 'relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
history_2=model_2.fit(x1_train,y_train, epochs=100, batch_size=10,validation_data=(x_test, y_test), verbose=1)

In [ ]:
loss_2,acc_2=model_2.evaluate(x1_test,y_test)
print("Model 2 Accuracy:", acc_2)

In [ ]:
from keras.layers import Dropout

In [ ]:
best_model = Sequential()

from keras.layers import Dropout

best_model.add(Dense(32, activation='relu', input_shape=(x1_train.shape[1],)))
best_model.add(Dropout(0.3))

best_model.add(Dense(16, activation='relu'))
best_model.add(Dropout(0.2))

best_model.add(Dense(8, activation='relu'))
best_model.add(Dense(1, activation='sigmoid'))

#COMPILE MODEL
best_model.compile(optimizer = 'adam',loss='binary_crossentropy', metrics=['accuracy'])

#EARLY STOPPING
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(
    monitor = 'val_loss',
    patience = 10,
    restore_best_weights = True
)

# FIT BEST MODEL

history=best_model.fit(x1_train, y_train, epochs=100,
                   batch_size=10, validation_data=(x_test,y_test),
                   callbacks =[early_stop], verbose=1)

#MODEL EVALUTION

loss_best, acc_best =best_model.evaluate(x_test, y_test)
print("Best Model Accuracy:", acc_best)